In [2]:
import pandas as pd
from data_apis import MyMarketIndexList, MyAlpacaStock, MyAlpacaJob, MyIsharesETF
from alpaca.data.timeframe import TimeFrameUnit
from alpaca.data.enums import Adjustment

In [2]:
# Get list of indices from EODHD
from numpy import empty

my_size_indices = MyMarketIndexList()
my_size_indices.set_selected_indices(['GSPC', 'MID', 'SML'])
#selected_indices_df = my_size_indices.get_selected_indices()
#historical_constituents = my_size_indices.get_selected_constituents()

In [4]:
my_job = MyAlpacaJob(my_index_list=my_size_indices, start='2007-01-01', end='2025-10-22', quantity=30,
                     unit=TimeFrameUnit.Minute, adjustment=Adjustment.RAW)
my_job.save_to_folder(limit = 10000)

Error: Invalid symbol '--'. No data retrieved.
No data to save for symbol --.
Data for symbol A already exists in C:/Users/David Trade/my_data/automatic/30Min_history/A.csv. Skipping download.
Data for symbol AA already exists in C:/Users/David Trade/my_data/automatic/30Min_history/AA.csv. Skipping download.
Saved data for symbol AAL to C:/Users/David Trade/my_data/automatic/30Min_history/AAL.csv
Saved data for symbol AAN to C:/Users/David Trade/my_data/automatic/30Min_history/AAN.csv
Saved data for symbol AAOI to C:/Users/David Trade/my_data/automatic/30Min_history/AAOI.csv
Saved data for symbol AAON to C:/Users/David Trade/my_data/automatic/30Min_history/AAON.csv
Saved data for symbol AAP to C:/Users/David Trade/my_data/automatic/30Min_history/AAP.csv
Saved data for symbol AAPL to C:/Users/David Trade/my_data/automatic/30Min_history/AAPL.csv
Saved data for symbol AAT to C:/Users/David Trade/my_data/automatic/30Min_history/AAT.csv
Saved data for symbol AAWW to C:/Users/David Trade/my_

In [ ]:
my_ishares_etf = MyIsharesETF(symbol="IWB", file_date="2025-10-24")
df = my_ishares_etf.load_from_xls()
my_job2 = MyAlpacaJob(start='2007-01-01', end='2025-10-22', quantity=30, unit=TimeFrameUnit.Minute, stock_list=df['Ticker'].tolist())
my_job2.save_to_folder(limit = 10000)

In [3]:
stock = MyAlpacaStock(symbol="AAPL")
stock.query_historical_data(start='2007-01-01', end='2025-10-26', quantity=30, unit=TimeFrameUnit.Minute, adjustment=Adjustment.RAW)
df = stock.get_df()

-
-
-

In [ ]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.enums import DataFeed, Adjustment
import requests
from zoneinfo import ZoneInfo
from datetime import datetime
from config import ConnectionParameters

In [3]:
# Set up connection parameters
cp = ConnectionParameters()
url_indices_list = f"https://eodhd.com/api/mp/unicornbay/spglobal/list?api_token={cp.EODHD_API_TOKEN}"
url_constituents = f"https://eodhd.com/api/mp/unicornbay/spglobal/comp/$index$?fmt=json&api_token={cp.EODHD_API_TOKEN}"

# Fetch the list of indices
response = requests.get(url_indices_list)

# ensure df_indices is always defined to avoid UnboundLocalError
df_indices = pd.DataFrame()
if response.status_code == 200:
    data = response.json()
    df_indices = pd.DataFrame(data)
else:
    print("Error fetching indices list:", response.status_code, response.text)

# Initialize an empty DataFrame to hold all historical constituents
all_hist = pd.DataFrame()

# Define size indices and fetch their historical constituents
size_indices = {'SP500':'GSPC', 'SP400':'MID', 'SP600':'SML'}
for index_name, index_ticker in size_indices.items():
    # Check if df_indices is defined, not empty and contains the index ticker
    if df_indices.empty:
        print(f"Skipping {index_name}: df_indices is empty")
        continue
    match = df_indices[df_indices['Code'] == index_ticker]
    if match.empty:
        print(f"Index ticker {index_ticker} not found in df_indices; skipping {index_name}")
        continue
    
    # Get the index symbol from the matched DataFrame
    index_symbol = match['ID'].iloc[0]

    # Fetch historical constituents for the index
    url = url_constituents.replace('$index$', index_symbol)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        hist_dict = data.get("HistoricalTickerComponents", {})
        if hist_dict:
            hist = pd.DataFrame(hist_dict.values())
            hist['size_index'] = index_name
            all_hist = pd.concat([all_hist, hist], ignore_index=True)
        else:
            print(f"No HistoricalTickerComponents for {index_name} ({index_ticker})")
    else:
        print(f"Error fetching constituents for {index_name}: {response.status_code} {response.text}")

# In all_hist dataframe when initial date is null set to 1900-01-01 and for end date null set to 2100-01-01

tz = ZoneInfo("America/New_York")

# Convert columns to datetime first (without timezone)
all_hist['StartDate'] = pd.to_datetime(all_hist['StartDate'], errors='coerce')
all_hist['EndDate']   = pd.to_datetime(all_hist['EndDate'], errors='coerce')

# Fill nulls and localize to NY timezone
all_hist['StartDate'] = all_hist['StartDate'].fillna(pd.Timestamp('1900-01-01')).dt.tz_localize(tz)
all_hist['EndDate']   = all_hist['EndDate'].fillna(pd.Timestamp('2100-01-01')).dt.tz_localize(tz)

# Create a range of business days (Mon–Fri) from 2012-04-04 to today
date_range = pd.date_range(
    start=datetime(2012, 4, 4, tzinfo=tz),
    end=datetime.now(tz),
    freq='B'
)

# Build final_df efficiently
records = []
for current_date in date_range:
    active = all_hist[
        (all_hist['StartDate'] <= current_date) &
        (all_hist['EndDate'] > current_date)
    ].copy()

    if active.empty:
        continue

    active.insert(0, 'date', current_date)
    records.append(active)

final_df = pd.concat(records, ignore_index=True) if records else pd.DataFrame()




In [4]:
# Count how many tickers are in each date and size_index
count_df = final_df.groupby(['date', 'size_index']).size().reset_index(name='constituent_count')
count_df.describe()

,constituent_count
count,10611.000000
mean,489.625012
std,79.751319
min,373.000000
25%,400.000000
50%,500.000000
75%,565.000000
max,604.000000


In [ ]:
# Clear all variables from previous cells
#del all_hist, active_constituents, temp_df, data, date_range, df_indices, hist, date_df, count_df
#del hist_dict, match, response, size_indices, index_name, index_symbol, index_ticker, url, url_constituents, url_indices_list

# Load fina_df from a csv file if needed
cp = ConnectionParameters()
sym_by_index = pd.read_csv('../my_data/symbol by index from 2012-04-04.csv', parse_dates=['date'])

#Remove from final_df the _old or _old1 suffix contained in some of the tickers (Code Column)
sym_by_index['Code'] = sym_by_index['Code'].str.replace('_old2', '', regex=False)
sym_by_index['Code'] = sym_by_index['Code'].str.replace('_old1', '', regex=False)
sym_by_index['Code'] = sym_by_index['Code'].str.replace('_old', '', regex=False)


#Replace - with . in the tickers (Code Column)
sym_by_index['Code'] = sym_by_index['Code'].str.replace('-', '.', regex=False)

# Loop through final_df and for each date retreive the ticker information with Alpaca API
alpaca_client = StockHistoricalDataClient(cp.alpaca_api_key, cp.alpaca_secret_key)
cont = 1
for current_date in sym_by_index['date'].dt.date.unique():
    # Filter constituents for the current date
    if cont == 1:
        constituents_on_date = sym_by_index[sym_by_index['date'] == current_date]
        #tickers = constituents_on_date['Code'].tolist()
        tickers = ["AAPL","MSFT","GOOGL"]  # For testing purposes, limit to a few tickers
        
        if not tickers:
            continue
        
        # Define the time range for the current date
        start_date = current_date.replace(tzinfo=ZoneInfo("America/New_York"))
        end_date = start_date + timedelta(days=1)
        
        # Create the bars request
        request = StockBarsRequest(
            symbol_or_symbols=tickers,
            timeframe=TimeFrame(amount=30, unit=TimeFrameUnit.Minute),  # type: ignore
            start=start_date,
            end=end_date,
            adjustment=Adjustment.ALL,  # type: ignore
            feed=DataFeed.SIP,  # type: ignore
        )
        
        # Fetch the bars data
        df_30m = alpaca_client.get_stock_bars(request).df
    cont += 1
    
    